In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from google.colab import drive

# FIFA 22 선수분석
# 값어치(value_eur)가 제일 높은 선수 20명 
PATH = "archive/"
FIFA_22 = pd.read_csv(PATH + 'players_22.csv', encoding='utf-8')
FIFA_22_value = FIFA_22.sort_values(by='value_eur', ascending=False).copy()
FIFA_22_value.head(20)

FIFA_22.shape
# 필요한 데이터만 추출
data = FIFA_22[['short_name','age','height_cm','weight_kg','nationality_name','club_name','overall','potential',
        'value_eur','wage_eur','player_positions','preferred_foot','international_reputation',
        'skill_moves', 'work_rate']]
data.isnull().sum() # 결측치 확인
data.dropna()   # 결측치 제거

# 오버롤 : 포지션 능력치의 평균치 / min, max
min(data.overall), max(data.overall)

# 국가별 선수 숫자
data['nationality_name'].value_counts()

# 모든 선수들의 주발
data.groupby(['preferred_foot']).count()[['short_name']]


/var/folders/d4/sthv_klj36vblf3ghqbtf9br0000gn/T/ipykernel_44329/1278826064.py:9: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  FIFA_22 = pd.read_csv(PATH + 'players_22.csv', encoding='utf-8')


,short_name
preferred_foot,
Left,4565
Right,14674
